In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))

In [4]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3T
import numpy as np
import pandas as pd

In [5]:
DIR = BASE_DIR/'scripts_res'/'Case3TAC3'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res/Case3TAC3')

In [6]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3T(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

In [ ]:
**

In [8]:
obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

strategy = 1
nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

for i in range(10):
    optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=2,random_state=i)
    try:
        optimizer.maximize(init_points=100, n_iter=200,p_hyper=[[4, 2.5, 0.78, 0.95,1e-5]],
                   acquisition_function=acquisition_function,dir=DIR/f'Case3T_{i}_Strategy{strategy}.csv', strategy=strategy)#
        print(optimizer.max)
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        x_3_list.extend([optimizer.max['params']['x3']])
        x_4_list.extend([optimizer.max['params']['x4']])
        x_5_list.extend([optimizer.max['params']['x5']])  
        obj_list.extend([optimizer.max['target']])                                  
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        con_4_list.extend([optimizer.max['constraint'][3]])
        all_res = [item['target'] for item in optimizer.res]
        all_fea = [item['allowed'] for item in optimizer.res]
        modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
        all_res_list.append(modified_res)    
    except Exception as e:
        print(e)
        obj_list.append(None)
        x_1_list.append(None)
        x_2_list.append(None)
        x_3_list.append(None)
        x_4_list.append(None)
        x_5_list.append(None)                        
        con_1_list.append(None)
        con_2_list.append(None)
        con_3_list.append(None)
        con_4_list.append(None) 
        modified_res = [-np.inf for _ in range(100+200)]
        all_res_list.append(modified_res)   
df2 = pd.DataFrame(all_res_list, columns=[list(range(100+200))])               
df = pd.DataFrame({
'obj': obj_list,
'x_1': x_1_list,
'x_2': x_2_list,
'x_3': x_3_list,
'x_4': x_4_list,
'x_5': x_5_list,
'con_1': con_1_list,
'con_2': con_2_list,
'con_3': con_3_list,
'con_4': con_4_list
})
df.to_csv(DIR/f'Case3T_{i}_Strategy{strategy}_Results.csv', index=False)
df2.to_csv(DIR/f'Case3T_{i}_Strategy{strategy}_Results2.csv', index=False)

|   iter    |  target   |  allowed  |    x1     |    x2     |    x3     |    x4     |    x5     |
-------------------------------------------------------------------------------------------------
| 1         | -21.36    | False     | -0.495    | -12.23    | -1.5      | 5.44      | 14.54     |
| 2         | -21.2     | False     | 9.705     | -7.61     | 9.3       | 15.04     | 3.645     |
| 3         | -16.65    | False     | -4.575    | 7.09      | 1.7       | -1.28     | -2.625    |
| 4         | -20.35    | False     | 4.435     | 18.85     | -7.1      | 5.12      | 0.345     |
| 5         | -21.2     | False     | 7.835     | 10.45     | 4.5       | -11.2     | -14.18    |
| 6         | -18.22    | False     | 10.89     | 16.75     | -9.1      | 3.84      | 14.87     |
| 7         | -21.47    | False     | -2.195    | -17.69    | -0.5      | 3.52      | -10.54    |
| 8         | -21.46    | False     | 7.495     | -13.49    | 2.9       | 14.4      | -9.885    |
| 9         | -20.21

In [7]:
df = pd.read_csv(DIR/'Case3T_9_Strategy1_Results.csv')
df.describe()

,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,-4.428478,1.437682,-0.903066,0.933446,2.570903,0.001866,0.849261,0.553058,1.237101,0.759256
std,5.991779,2.768361,2.699788,3.277410,5.275510,3.793341,1.006224,1.149248,1.279565,0.469993
min,-18.198143,-2.214486,-5.965186,-3.988715,-2.008186,-5.947014,-0.669594,-0.849640,-0.062368,0.144447
25%,-4.905977,-0.956262,-1.986041,0.078040,-0.988248,-1.976926,0.111074,-0.367419,0.328849,0.349882
50%,-4.438311,1.987168,-0.803776,0.165657,1.946520,-0.015100,1.240735,0.798057,0.571748,0.833906
75%,-1.028639,2.057239,1.077813,3.946805,2.071284,1.137960,1.693183,1.147311,2.774139,1.133516
max,2.837060,5.961188,1.992082,5.992287,15.095042,6.979299,1.863388,2.523193,2.966808,1.477040
